# importing all the necessary libraries

In [32]:

import pandas as pd
import requests
import nltk
from nltk import sent_tokenize as sent_t
from nltk.corpus import stopwords as st
from nltk import word_tokenize as wt
import re
import heapq


### featching data from Excel file

In [40]:
df=pd.read_excel('TASK.xlsx')
# droping the unuseful row 
df.drop([0],inplace=True)
df

,TEST DATASET,Unnamed: 1
1,NaN,Acnesol Gel is an antibiotic that fights bacte...
2,NaN,Ambrodil Syrup is used for treating various re...
3,NaN,Augmentin 625 Duo Tablet is a penicillin-type ...
4,NaN,Azithral 500 Tablet is an antibiotic used to t...
5,NaN,Alkasol Oral Solution is a medicine used in th...
...,...,...
996,NaN,Azapure Tablet belongs to a group of medicines...
997,NaN,Arimidex 1mg Tablet is used alone or with oth...
998,NaN,Arpimune ME 100mg Capsule is used to prevent y...
999,NaN,Amlodac CH Tablet is a combination medicine us...


### Renaming the Columns of DataFrame

In [41]:

df.columns=['summary','TEXT DATA']
df

,summary,TEXT DATA
1,NaN,Acnesol Gel is an antibiotic that fights bacte...
2,NaN,Ambrodil Syrup is used for treating various re...
3,NaN,Augmentin 625 Duo Tablet is a penicillin-type ...
4,NaN,Azithral 500 Tablet is an antibiotic used to t...
5,NaN,Alkasol Oral Solution is a medicine used in th...
...,...,...
996,NaN,Azapure Tablet belongs to a group of medicines...
997,NaN,Arimidex 1mg Tablet is used alone or with oth...
998,NaN,Arpimune ME 100mg Capsule is used to prevent y...
999,NaN,Amlodac CH Tablet is a combination medicine us...


## Preprocessing

In [42]:
# converting the entire paragraph into lower case
df['summary']=df['TEXT DATA'].str.lower()
# Reseting index
df=df.reset_index()
# droping unwanted columns
df=df.drop(['index'],axis=1)
df

,summary,TEXT DATA
0,acnesol gel is an antibiotic that fights bacte...,Acnesol Gel is an antibiotic that fights bacte...
1,ambrodil syrup is used for treating various re...,Ambrodil Syrup is used for treating various re...
2,augmentin 625 duo tablet is a penicillin-type ...,Augmentin 625 Duo Tablet is a penicillin-type ...
3,azithral 500 tablet is an antibiotic used to t...,Azithral 500 Tablet is an antibiotic used to t...
4,alkasol oral solution is a medicine used in th...,Alkasol Oral Solution is a medicine used in th...
...,...,...
995,azapure tablet belongs to a group of medicines...,Azapure Tablet belongs to a group of medicines...
996,arimidex 1mg tablet is used alone or with oth...,Arimidex 1mg Tablet is used alone or with oth...
997,arpimune me 100mg capsule is used to prevent y...,Arpimune ME 100mg Capsule is used to prevent y...
998,amlodac ch tablet is a combination medicine us...,Amlodac CH Tablet is a combination medicine us...


In [43]:
# calculating stopwords to be ignored while dealing with sentences
stopwords=st.words('english')
# find_Summary() func is for preprocessing the individual paragraph out of 1000 para's 
#and returning a particular summarized text.
def find_Summary(para):
    # regex operation for removing numbers, special characters and  whitespaces
    art=re.sub(r'\s+', ' ', para)
    article=re.sub('[^a-zA-Z]', ' ', para)
    new_article=re.sub(r'\s+', ' ', article)
    
    # breaking of the paragraph into sentences
    sentences=nltk.sent_tokenize(art)
    sentences=[i.rstrip('.') for i in sentences]
    print(sentences)
    word_freq = {}
    # converting para into word tookens and finding frequency of each word
    for word in nltk.word_tokenize(new_article):
        
        if word not in stopwords:
            if word not in word_freq.keys():
                word_freq[word]=1
            else:
                word_freq[word]+=1
                
    # dividing each word frequency with max_word's frequency
    maximum_freq=max(word_freq.values())
    for word in word_freq.keys():
        word_freq[word]=word_freq[word]/maximum_freq
        
    sentence_scores = {}
    # finding the sentence frequency on basis of word frequency
    
    for sent in sentences:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_freq.keys():
                if len(sent.split(' ')) < 35:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_freq[word]
                    else:
                        sentence_scores[sent] += word_freq[word]   
                        
    #finding the top four sentences on basis of sentences score in an order of decreasing to form a summary.
    summary_sentences = heapq.nlargest(4, sentence_scores, key=sentence_scores.get)
    # joining the sentences for summary formation
    summary = ' '.join(summary_sentences)                    
    return summary

In [44]:

for i in range(len(df)):
    
    df.loc[i]['summary']=find_Summary(df.loc[i]['summary'])


['acnesol gel is an antibiotic that fights bacteria', 'it is used to treat acne, which appears as spots or pimples on your face, chest or back', 'this medicine works by attacking the bacteria that cause these pimples.acnesol gel is only meant for external use and should be used as advised by your doctor', 'you should normally wash and dry the affected area before applying a thin layer of the medicine', 'it should not be applied to broken or damaged skin', 'avoid any contact with your eyes, nose, or mouth', 'rinse it off with water if you accidentally get it in these areas', 'it may take several weeks for your symptoms to improve, but you should keep using this medicine regularly', 'do not stop using it as soon as your acne starts to get better', 'ask your doctor when you should stop treatment.common side effects like minor itching, burning, or redness of the skin and oily skin may be seen in some people', 'these are usually temporary and resolve on their own', 'consult your doctor if t

['allegra 180mg tablet belongs to a group of medicines called antihistamines', 'it is used to treat various allergic conditions such as hay fever, conjunctivitis and some skin reactions such as eczema, hives, and reactions to bites and stings', 'it relieves watery eyes, runny nose, sneezing, and itching.allegra 180mg tablet should be taken empty stomach', 'the dose may vary depending on what you are taking it for', 'take it as it has been prescribed by your doctor', 'you may need this medicine only on days you have symptoms, or you may need to take it every day to prevent symptoms from happening', 'if you stop taking it earlier than advised, your symptoms may come back.this medicine is generally very safe', 'the most common side effects include headache, drowsiness, dizziness, and nausea', 'these are usually mild and go away after a couple of days as your body adjusts', 'consult your doctor if any of the side effects persist or worry you.before taking it, you should tell your doctor if

['amoxyclav 625 tablet is a penicillin-type of antibiotic that helps your body fight infections caused by bacteria', 'it is used to treat infections of the lungs (e.g., pneumonia), ear, nasal sinus, urinary tract, skin and soft tissue', 'it will not work for viral infections such as the common cold.amoxyclav 625 tablet is best taken with a meal to reduce the chance of a stomach upset', 'you should take it regularly at evenly spaced intervals as per the schedule prescribed by your doctor', 'taking it at the same time every day will help you to remember to take it', 'the dose will depend on what you are being treated for, but you should always complete a full course of this antibiotic as prescribed by your doctor', 'do not stop taking it until you have finished, even when you feel better', 'if you stop taking it early, some bacteria may survive and the infection may come back or worsen.the most common side effects of this medicine include vomiting, nausea, and diarrhea', 'these are usual

['allercet-dc tablet is a combination medicine used in the treatment of allergy symptoms like runny nose, sneezing, throat irritation, watery eyes and congestion or stuffiness in the nose.allercet-dc tablet is taken with or without food in a dose and duration as advised by the doctor', 'the dose you are given will depend on your condition and how you respond to the medicine', 'you should keep taking this medicine for as long as your doctor recommends', 'if you stop treatment too early your symptoms may come back and your condition may worsen', 'let your healthcare team know about all other medications you are taking as some may affect, or be affected by this medicine.the most common side effects are dryness in the mouth, headache, nausea, vomiting, and tiredness', 'most of these are temporary and usually resolve with time', 'contact your doctor straight away if you are at all concerned about any of these side effects', 'it can also cause dizziness and sleepiness, so do not drive or do 

['azax 500 tablet is an antibiotic used to treat various types of bacterial infections of the respiratory tract, ear, nose, throat, lungs, skin, and eye in adults and children', 'it is also effective in typhoid fever and some sexually transmitted diseases like gonorrhea.azax 500 tablet is a broad-spectrum type of antibiotic effective in killing many types of gram-positive bacteria, some types of gram-negative bacteria and other microorganisms', 'this medicine is taken orally, preferably either one hour before or 2 hours after a meal', 'it should be used regularly at evenly spaced time intervals as prescribed by your doctor', 'do not skip any doses and finish the full course of treatment even if you feel better', 'stopping the medicine too early may lead to the return or worsening of the infection.commonly seen side effects seen with this medicine include vomiting, nausea, stomach pain, and diarrhea', 'these are usually temporary and subside with the completion of treatment', 'consult y

...

['apidra solostar 100iu/ml injection is used for the treatment of blood sugar control in people with type 1 and type 2 diabetes mellitus', 'it is a fast-acting type of insulin that helps to lower blood sugar levels after meals and reduce the chances of developing serious complications of diabetes.apidra solostar 100iu/ml injection is normally prescribed along with a long-acting type of insulin or other diabetes medicines', 'your doctor or nurse will teach you the correct way of injecting it under the skin', 'it should be taken 15 minutes before a meal or within 20 minutes after starting a meal', 'do not stop taking it unless your doctor tells you to', 'it is only part of a treatment program that should also include a healthy diet, regular exercise, and weight reduction as advised by your doctor.check your blood sugar levels regularly, keep track of your results and share them with your doctor', 'this is very important to work out the correct dose of the medicine for you.the most common

['aceclorite plus 100 mg/500 mg tablet is a pain relieving medicine', 'it is used to reduce pain and inflammation in conditions like rheumatoid arthritis, ankylosing spondylitis, and osteoarthritis', 'it may also be used to relieve muscle pain, back pain, toothache, or pain in the ear and throat.aceclorite plus 100 mg/500 mg tablet should be taken with or without food', 'you should take it regularly as advised by your doctor', 'your doctor may change the dose and time between doses according to your pain level and your needs', 'do not take more or use it for a longer duration than recommended by your doctor.some of the common side effects of this medicine include nausea, vomiting, stomach pain, loss of appetite, heartburn, and diarrhea', 'if any of these side effects bother you or do not go away with time, you should let your doctor know', 'your doctor may help you with ways to reduce or prevent these side effects by prescribing an alternative medicine or adjusting the dose.the medicin

['anxozap 15 tablet is a prescription medicine that relieves symptoms of anxiety such as irritability, restlessness, lack of concentration, fatigue, sweating, increased heart rate, and unwanted or racing thoughts', 'it may also be used in alcoholics to treat anxiety associated with alcohol withdrawal.anxozap 15 tablet belongs to a class of medicines called benzodiazepines', 'it may be taken with or without food', 'however, take it at the same time each day as this helps to maintain a consistent level of medicine in the body', 'take this medicine in the dose and duration as advised by your doctor as it has a high potential of habit-forming.if you have missed a dose, take it as soon as you remember and finish the full course of treatment even if you feel better', 'it is important that this medication is not stopped suddenly without talking to your doctor as it may cause nausea and anxiety', 'it should only be used for a limited period as long term use of this medicine is not recommended.

['amnurite 25 tablet sr is a combination of two medicines used to treat neuropathic pain', 'it works by protecting the nerve fibers against damage, repairing the damaged nerves', 'this way it effectively relieves the pain sensation and also elevates the mood.amnurite 25 tablet sr may be taken with or without food', 'however, it is advised to take it at the same time each day as this helps to maintain a consistent level of medicine in the body', 'take this medicine in the dose and duration advised by your doctor', 'if you miss a dose of this medicine, take it as soon as you remember', 'finish the full course of the treatment even if you feel better', 'it is important not to stop taking this medicine suddenly.some common side effects of this medicine include constipation, dry mouth, difficulty in urination, drowsiness, blurred vision, and increased heart rate', 'it may also cause dizziness and sleepiness, so do not drive or do anything that requires mental focus until you know how this m

['amlodac ch tablet is a combination medicine used in the treatment of hypertension (high blood pressure)', 'it also helps to reduce the risk of stroke and heart attack', 'it should be taken in the morning with food to avoid frequent urination at night.amlodac ch tablet should be taken with food preferably in the morning to avoid frequent urination at night', 'keep taking it for as long as advised by your doctor', 'even if you feel well, do not stop this medicine on your own because high blood pressure often has no symptoms', 'if you stop taking it, your condition may get worse', 'keeping active with regular exercise, reducing your weight and eating a healthy diet will also help control your blood pressure', 'follow your doctor’s advice while taking this medicine.glucose intolerance, ankle swelling, headache, and decreased potassium level are some common side effects of this medicine', 'consult your doctor if any of these bother you, or get worse, or do not go away', 'drink plenty of f

## exporting summary data

In [49]:
#exporting summary data into  excel file named summary.xlsx
data_summary=pd.Series(df['summary'])
data_summary.to_excel('summary.xlsx')
#displaying summary data 
data_summary[0]

'ask your doctor when you should stop treatment.common side effects like minor itching, burning, or redness of the skin and oily skin may be seen in some people consult your doctor if they bother you or do not go away.it is a safe medicine, but you should inform your doctor if you have any problems with your bowels (intestines) also, inform the doctor if you have ever had bloody diarrhea caused by taking antibiotics or if you are using any other medicines to treat skin conditions this medicine works by attacking the bacteria that cause these pimples.acnesol gel is only meant for external use and should be used as advised by your doctor'

In [58]:
df.iloc[6][1]

'Ascoril LS Syrup is a combination medicine used in the treatment of cough with mucus. It thins mucus in the nose, windpipe and lungs, making it easier to cough out. It also provides relief from runny nose, sneezing, itching, and watery eyes.Ascoril LS Syrup is taken with or without food in a dose and duration as advised by the doctor. The dose you are given will depend on your condition and how you respond to the medicine. You should keep taking this medicine for as long as your doctor recommends. If you stop treatment too early your symptoms may come back and your condition may worsen. Let your healthcare team know about all other medications you are taking as some may affect, or be affected by this medicine.The most common side effects are nausea, diarrhea, stomach pain, vomiting, muscle cramp, headache, skin rash, and increased heart rate. Most of these are temporary and usually resolve with time. Contact your doctor straight away if you are at all concerned about any of these side